In [ ]:
# A simple program to check if GPS errors exist in DWR-G HXV files

using CSV
using Dates, DataFrames
using NativeFileDialog
using Printf


function get_displacements(arry)
#####################################
    
    displacements = []

    if length(arry[1]) == 3
    
        for i in arry
            append!(displacements,parse(Int, SubString.(i, 1, 1), base=16)*16^2 + parse(Int, SubString.(i, 2, 2), base=16)*16^1 + parse(Int, SubString.(i, 3, 3), base=16)*16^0)
        end
        
    else
        
        for i in arry
            append!(displacements,parse(Int, SubString.(i, 1, 1), base=16)*16^1 + parse(Int, SubString.(i, 2, 2), base=16)*16^0)
        end
        
    end

    displacements[findall(>=(2048), displacements)] = 2048 .- displacements[findall(>=(2048), displacements)];
    
    return(displacements./100)
    
    end     # get_displacements()


function get_HNW(infil)
#####################################
        
    global df = DataFrame(CSV.File(infil,header=0, delim=",", types=String));

    # Calculate sequence numbers
    arry = SubString.(df.Column1, 3, 4)

    global sequence = []

    for i in arry
        append!(sequence,parse(Int, SubString.(i, 1, 1), base=16)*16^1 + parse(Int, SubString.(i, 2, 2), base=16)*16^0)
    end

    # Calculate heave WSEs
    arry = SubString.(df.Column3, 1, 3);
    heave = get_displacements(arry);

    # Calculate north WSEs
    arry = SubString.(df.Column3, 4, ) .* SubString.(df.Column4, 1, 2)
    north = get_displacements(arry);

    # Calculate north WSEs
    arry = SubString.(df.Column4, 3, 4) .* SubString.(df.Column5, 1, 1)
    west = get_displacements(arry);

    return(heave, north, west)

    end    # get_HNW()


function calc_wse(infil, wse_df, start_date)
#####################################    
    
    heave, north, west = get_HNW(infil)
    
    # Identify any gaps in the recorded data
    tt = [0]
    append!(tt,diff(sequence))
    tt[tt.<0] .+= 256;
    tt1 = cumsum(tt);

    # only use the first 2304 points if more in df
    if length(tt1) > 2304
        tt1 = tt1[1:2304]
    end

    [wse_df[tt1[i]+1,2] = heave[i] for i in eachindex(tt1)];
    [wse_df[tt1[i]+1,3] = north[i] for i in eachindex(tt1)];
    [wse_df[tt1[i]+1,4] = west[i] for i in eachindex(tt1)];
    
    return(wse_df)
    
    end    # calc_wse()


function process_spectrum_file()
#####################################
    # determine whether data is from DWR-G buoy or not
    global is_gps = false
    sync_word_location = findall(x -> x == "7FFF", df.Column2)

    for j in sync_word_location

        i = df.Column2[j+1]
        word_number = parse(Int, SubString.(i, 1, 1), base=16)*16^0
        word = parse(Int, SubString.(i, 2, 2), base=16)*16^2 + parse(Int, SubString.(i, 3, 3), base=16)*16^1 + parse(Int, SubString.(i, 4, 4), base=16)*16^0

        # Test whether buoy is MkIII or DWR-G - see p.51 Table 5.7.5a. Organization and significance of the system file data 
        # If DWR-G:
        #     Av0 = 0; Ax0 = 0; Ay0 = 0; O = 0; and Inclination = 0
        
        if (word_number == 7 && word == 0)
            is_gps = true
        end

    end
    
    return(is_gps)
    
    end    # process_spectrum_file()


################################################
################################################
##           START OF MAIN PROGRAM
################################################
################################################

# Widen screen for better viewing
display("text/html", "<style>.container { width:100% !important; }</style>")

hxv_directory = pick_folder()

# build list of all hxv files in selected directory
hxv_files = filter(x->occursin(".hxv",x), readdir(hxv_directory));
hxv_files = hxv_files[findall(x->endswith(uppercase(x), ".HXV"), hxv_files)];

total_errors = 0

println("Reading ",hxv_directory," now")
flush(stdout)

for i in 1:length(hxv_files)
     
    infil = hxv_directory * "\\" * hxv_files[i]
    # extract the datetime from the file name
    date_str = split(infil,".")[1]
    ll = length(date_str)
    global start_date = DateTime.(date_str[ll-16:ll-1], "yyyy-mm-ddTHHhMMZ")

    # create df of 2304 rows, each 0.78s apart
    global wse_df = DataFrame(Date = unix2datetime.(datetime2unix.(start_date) .+ (0:1/1.28:1800-1/1.28)),
        Heave = zeros(2304), North = zeros(2304), West = zeros(2304));

    # populate the df based on sequence numbers
    wse_df = calc_wse(infil, wse_df, start_date)
    
    # identify if data is from DWR-G buoy
    is_gps = process_spectrum_file()
    
    # if DWR-G buoy then locate where gps errors occur
    
    if is_gps
    
        gps_errors = findall(isodd,parse.(Int,SubString.(string.(df.Column4), 2, 2), base = 16))
        
        error_count = length(gps_errors)
        total_errors += error_count
        
        if error_count > 0
            
            println(hxv_files[i],' ',error_count," GPS errors")
            
        end
        
    end
    
end    

if (is_gps == true) 
    
    if (total_errors == 0)
        
        println(hxv_directory," has no GPS errors!")
        
    else
    
    println("______________________________________")
        
    end

else
        
    println("Not a DWR-G buoy!")
        
end